## Napari Viewer
## Author: Pablo Siliceo Portugal (psiliceop@gmail.com)

### Downloading NAPARI

In [4]:
#Downloading Napari for the first time
conda create -y -n napari-env -c conda-forge python=3.9
conda activate napari-env
conda install -c conda-forge napari

#****Napari can also be installed with pip using
# python -m install "napari[all]"

SyntaxError: invalid syntax (635086583.py, line 2)

### Running the viewer

In [1]:
## Dependencies needed
try:
    import napari
    import pandas
    from napari.layers import Shapes
    from napari.utils.notifications import show_info
except:
    pass
import pandas as pd
import numpy as np
import random
import tifffile as tiff
import scimap as sm 
from tifffile import imread
import dask.array as da
from dask.cache import Cache
import zarr
import os
import matplotlib.patches as mpatches
import scipy.spatial.distance as sdistance
import ast
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure
from pathlib import Path
from magicgui import event_loop, magicgui
from PyQt5.QtWidgets import QMessageBox
import sys
from PyQt5.QtWidgets import QApplication, QPushButton
from PyQt5.QtCore import Slot
import enum
from dask_image.imread import imread as daskread
from napari.utils.notifications import show_info
#from joblib import Parallel, delayed

Running SCIMAP  2.1.3


2025-04-10 12:04:03.362092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-10 12:04:03.376703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-10 12:04:03.381621: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-10 12:04:03.392594: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import numpy as np

def circle_coordinates(cx: float, cy: float, radius: float, num_points: int = 100) -> np.ndarray:
    """Devuelve un array Nx2 con las coordenadas (x,y) de un polígono que dibuja un círculo."""
    angles = np.linspace(0, 2*np.pi, num_points)
    xs = cx + radius * np.cos(angles)
    ys = cy + radius * np.sin(angles)
    return np.vstack([xs, ys]).T


In [2]:
# This code will open the Napari viewer with the necessary widgets to open and perform operations on the image
viewer = napari.Viewer()

# Widgets
# Open the image in the viewer
@magicgui(call_button='Open image', layout='vertical')
def open_large_image(image_path=Path(), contrast_limit_txt=Path(), ab_list_path=Path()):
    ab = pd.read_csv(ab_list_path)
    ab = list(ab["ABS"])
    cl_txt = str(contrast_limit_txt)

    if cl_txt == '.':
        image = tiff.TiffFile(image_path, is_ome=False)
        z = zarr.open(image.aszarr(), mode='r')
        # Identify the number of levels in the pyramid
        n_levels = len(image.series[0].levels)

        # Check if there are pyramids available
        if n_levels > 1:
            pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
            multiscale = True
        else:
            pyramid = da.from_zarr(z)
            multiscale = False


        viewer.add_image(
            pyramid, multiscale=multiscale, channel_axis=0, name=ab, visible=False)

    else:
        with open(cl_txt, 'r') as cl:
            c = cl.read()
        contrast_limit = ast.literal_eval(c)

        image = tiff.TiffFile(image_path, is_ome=False)
        z = zarr.open(image.aszarr(), mode='r')
        n_levels = len(image.series[0].levels)

        if n_levels > 1:
            pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
            multiscale = True
        else:
            pyramid = da.from_zarr(z)
            multiscale = False

        viewer.add_image(
            pyramid, multiscale=multiscale, channel_axis=0, name=ab, visible=False, contrast_limits=contrast_limit)

# Open the segmentation mask in the viewer
@magicgui(call_button='Open mask', layout='vertical')
def open_mask(mask_path=Path()):
    seg_m = tiff.imread(mask_path)
    if (len(seg_m.shape) > 2) and (seg_m.shape[0] > 1):
        seg_m = seg_m[0]
    viewer.add_labels(seg_m, name='MASK')

# Load previously created shapes into the viewer
@magicgui(call_button='Load Shapes', layout='vertical', shapes_path={"mode": "d"})
def load_shapes(shapes_path: Path):
    shapes_path = str(shapes_path) + "/"
    shapes_list = os.listdir(shapes_path)
    names = []
    for filename in shapes_list:
        name = filename.replace(".txt", "")
        names.append(name)
        with open(shapes_path + filename, 'r') as f:
            shapes_str = f.read()
        shapes_str = shapes_str.replace('\n', '').replace('      ', '').replace('array(', '').replace(')', '')
        shapes = ast.literal_eval(shapes_str)
        shape_arrays = [np.array(s) for s in shapes]
        viewer.add_shapes(shape_arrays, shape_type='polygon', edge_width=0,
                          edge_color='#777777ff', face_color='white', name=name)

# Save the contrast limits applied to the image
@magicgui(call_button='Save contrast limits', layout='vertical', output_file={"mode": "d"})
def save_contrast_limits(output_file: Path, ab_list_path=Path(), name=""):
    contrast_limit = []
    ab = pd.read_csv(ab_list_path)
    ab = list(ab["ABS"])
    for antibody in ab:
        contrast_limit.append(viewer.layers[antibody].contrast_limits)

    with open(str(output_file) + "/" + name + ".txt", "w") as output:
        output.write(str(contrast_limit))

# Save a shape drawn on the image (coordinate array as a .txt file)
@magicgui(call_button='Save shape array', layout='vertical', output_file={"mode": "d"})
def save_shapes(output_file: Path, shape_name=""):
    shapes = viewer.layers[shape_name].data
    with open(str(output_file) + "/" + shape_name + ".txt", 'w') as output:
        output.write(str(shapes))

# Get a CSV file of the cell IDs in a selected shape
@magicgui(call_button='Cut and Save ROIs', filepath={"mode": "d"})
def cut_mask(filepath: Path, shape_name=""):
    if 'MASK' not in viewer.layers:
        show_info('No mask layer named "MASK" was found.')
        return
    if shape_name not in viewer.layers:
        show_info(f'No shape layer named "{shape_name}" was found.')
        return

    mask_to_cut = viewer.layers['MASK'].data
    shape = mask_to_cut.shape
    selected_area = viewer.layers[shape_name].to_labels(labels_shape=shape)
    removable_cells = []
    for i in range(mask_to_cut.shape[0]):
        for j in range(mask_to_cut.shape[1]):
            cell = mask_to_cut[i, j]
            if selected_area[i, j] > 0 and cell not in removable_cells and cell > 0:
                removable_cells.append(cell)
    df = pd.DataFrame({'cellid': removable_cells})
    df = df.astype(int)
    df.to_csv(str(filepath) + '/' + shape_name + '_selected_cell_ids.csv', index=False)

# Close all layers loaded in the viewer
@magicgui(call_button='Close all', layout='vertical')
def close_all():
    viewer.layers.clear()

#Viewing any metadata annotation by mapping points in cells
@magicgui(call_button = 'View metadata', layout='vertical')
def view_metadata(adata_path=Path(),image_name = "", metadata_column = ""):
    path = str(adata_path)
    adata = sm.pp.mcmicro_to_scimap(path,remove_dna=False, remove_string_from_name=None, log=False,
                                random_sample=None, CellId='CellID', split='X_centroid',
                                custom_imageid=None, min_cells=None, output_dir=None)
    adata = adata[adata.obs['imageid'] == image_name]
    available_phenotypes = list(adata.obs[metadata_column].unique())
    for i in available_phenotypes:
        coordinates = adata[adata.obs[metadata_column] == i]
        coordinates = pd.DataFrame({'y': coordinates.obs["Y_centroid"] ,'x': coordinates.obs["X_centroid"]})
        # points = coordinates.values.tolist()
        points = coordinates.values
        r = lambda: random.randint(0, 255)  # random color generator
        point_color = '#%02X%02X%02X' % (r(), r(), r())  # random color generator

        # check if point_color is a dict and if so isolate the color to the specific categoty
        phenotype_layer = viewer.add_points(points,size=10,face_color=point_color,visible=False,name=i)



#Counting selected cells

@magicgui(call_button='Count Cells', layout='vertical')
def count_selected_cells(shape_name: str, cell_info_csv: Path):
    """
    Cuenta las células dentro de un Shape en Napari usando solo el archivo de cuantificación (CSV).

    Parámetros:
    - shape_name: Nombre de la capa de shapes en Napari.
    - cell_info_csv: Archivo CSV con la información de todas las células.
    """
    # Verificar que la capa de shapes existe en Napari
    if shape_name not in [layer.name for layer in viewer.layers]:
        show_info(f'No shape layer named "{shape_name}" was found.')
        return

    # Obtener la capa de shapes
    shape_layer = viewer.layers[shape_name]

    # Leer el archivo de cuantificación
    try:
        cell_info_df = pd.read_csv(cell_info_csv)
    except Exception as e:
        show_info(f'Error reading the cell information file: {e}')
        return

    # Verificar las columnas de coordenadas en el CSV
    x_col, y_col = None, None
    for col_x, col_y in [('X_centroid', 'Y_centroid'), ('x', 'y'), ('X', 'Y')]:
        if col_x in cell_info_df.columns and col_y in cell_info_df.columns:
            x_col, y_col = col_x, col_y
            break
    if x_col is None or y_col is None:
        show_info('No valid coordinate columns found in the cell information file.')
        return

    # Obtener la primera región de interés (ROI) del shape seleccionado
    shapes = shape_layer.data
    roi_polygon = shapes[0]  # Se asume que el usuario selecciona una sola ROI

    # Convertir coordenadas del CSV en un array de puntos
    cell_coords = np.column_stack((cell_info_df[x_col], cell_info_df[y_col]))

    # Verificar qué células están dentro del Shape
    from matplotlib.path import Path
    polygon_path = Path(roi_polygon)
    mask = polygon_path.contains_points(cell_coords)

    # Contar las células dentro del Shape
    cell_count = np.sum(mask)

    # Mostrar el número total de células dentro del Shape
    show_info(f'Total cells within "{shape_name}": {cell_count}')
    print(f'Total cells within "{shape_name}": {cell_count}')




# Counting selected cells and extract their information to a .csv file
@magicgui(call_button='Save cells in selected ROI', layout='vertical', output_csv={"mode": "d"})
def save_selected_cells(
    output_csv: Path,
    shape_name: str = "",
    cell_info_csv: Path = Path(),
    output_file_name: str = ""
):
    # Verificar si existen la capa MASK y la capa de shapes
    if 'MASK' not in viewer.layers:
        show_info('No mask layer named "MASK" was found.')
        return
    if shape_name not in viewer.layers:
        show_info(f'No shape layer named "{shape_name}" was found.')
        return

    # Extraer la capa de la máscara y la capa de shapes
    mask_layer = viewer.layers['MASK']
    mask_data = mask_layer.data
    shape_layer = viewer.layers[shape_name]
    shape_data = shape_layer.to_labels(labels_shape=mask_data.shape)

    # Identificar las celdas que caen dentro del ROI
    overlapping_cells = mask_data[shape_data > 0]
    unique_cells = np.unique(overlapping_cells)
    unique_cells = unique_cells[unique_cells != 0]  # excluir ID 0
    cell_count = len(unique_cells)

    # Mostrar el número total de celdas en el ROI
    show_info(f'Total cells within "{shape_name}": {cell_count}')
    print(f'Total cells within "{shape_name}": {cell_count}')

    # Leer el CSV con la información de TODAS las celdas
    try:
        cell_info_df = pd.read_csv(cell_info_csv)
    except Exception as e:
        show_info(f'Error reading the cell information file: {e}')
        return

    # Verificar la columna que corresponde a CellID
    cell_id_column = None
    for col in ['cellid', 'CellID', 'cell_id', 'Cell_Id', 'cellID']:
        if col in cell_info_df.columns:
            cell_id_column = col
            break
    if cell_id_column is None:
        show_info('No cell ID column was found in the cell information file.')
        return

    # Filtrar la información de solo las celdas seleccionadas
    selected_cells_info = cell_info_df[cell_info_df[cell_id_column].isin(unique_cells)]

    # Guardar la información de las celdas seleccionadas en CSV
    try:
        selected_cells_info.to_csv(
            str(output_csv) + "/" + output_file_name + ".csv",
            index=False
        )
        show_info(
            f'Information on {cell_count} selected cells saved in {output_csv}'
        )
    except Exception as e:
        show_info(f'Error saving the selected cells file: {e}')




#Creating a Voronoi plot from a designated shape
@magicgui(call_button='Voronoi plot', layout='vertical', output_dir={"mode": "d"})
def voronoi_plot(output_dir: Path , adata_path=Path(), shape_name = "", image_name = "", cluster_name =  "", file_name = ""):
    path = str(adata_path)
    adata = sm.pp.mcmicro_to_scimap(path,remove_dna=False, remove_string_from_name=None, log=False,
                                    random_sample=None, CellId='CellID', split='X_centroid',
                                    custom_imageid=None, min_cells=None, output_dir=None)
    shapes = viewer.layers[shape_name].data
    shapes = shapes[0].tolist()
    x= shapes[0]
    y= shapes[2]
    x_1= x[1]
    x_2= y[1]
    y_1= x[0]
    y_2= y[0]
    n_colors = {0: "#D3D3D3" , 1: '#D3D3D3',2: "#D3D3D3", 3: "#FF0000", 4: "#D3D3D3",
                5: "#D3D3D3",6: '#D3D3D3', 7: "#FFD343", 8: "#D3D3D3", 9: "#D3D3D3"}
    sm.pl.voronoi(adata, color_by=cluster_name, x_coordinate='X_centroid', y_coordinate='Y_centroid', imageid='imageid',
                  subset = image_name, x_lim = [x_1,x_2], y_lim =[y_1,y_2], plot_legend = True, flip_y = True, 
                  overlay_points = cluster_name,voronoi_alpha = 0.7, voronoi_line_width = 0.3, overlay_point_size = 8, 
                  overlay_point_alpha = 1, legend_size = 15, overlay_points_colors = n_colors, colors = n_colors, 
                  fileName = file_name + ".pdf", saveDir = str(output_dir) + "/")


#########################################################################################

# Create a circle with n cells 
# pendiente mover dependencias

import numpy as np
import pandas as pd
from pathlib import Path
import napari
from magicgui import magicgui
from magicgui.widgets import PushButton
from napari.utils.notifications import show_info

def circle_coordinates(cx, cy, radius, num_points=100):
    angles = np.linspace(0, 2*np.pi, num_points, endpoint=False)
    x = cx + radius * np.cos(angles)
    y = cy + radius * np.sin(angles)
    return np.column_stack([x, y])

@magicgui(
    call_button='Crear círculo',
    layout='vertical',
    center_x={'min': -1e9, 'max': 1e9, 'step': 1},
    center_y={'min': -1e9, 'max': 1e9, 'step': 1},
    num_cells={'min': 1, 'max': 1e7, 'step': 1},
)
def create_circle_for_n_cells(
    cell_info_csv: Path = None,
    center_x: float = 0.0,
    center_y: float = 0.0,
    shape_name: str = "circle_auto",
    num_cells: int = 1000
):
    if cell_info_csv is None or not cell_info_csv.is_file():
        show_info(f"No se encontró el archivo CSV: {cell_info_csv}")
        return
    if 'viewer' not in globals():
        show_info("No existe un viewer de Napari activo llamado 'viewer'.")
        return
    
    try:
        df = pd.read_csv(cell_info_csv)
    except Exception as e:
        show_info(f"Error al leer el CSV: {e}")
        return

    possible_x_cols = ['X_centroid','x','X']
    possible_y_cols = ['Y_centroid','y','Y']
    
    x_col, y_col = None, None
    for c in possible_x_cols:
        if c in df.columns:
            x_col = c
            break
    for c in possible_y_cols:
        if c in df.columns:
            y_col = c
            break

    if x_col is None or y_col is None:
        show_info("No se encontraron columnas de coordenadas X, Y en el CSV.")
        return

    df['dist_to_center'] = np.sqrt((df[x_col] - center_x)**2 + (df[y_col] - center_y)**2)
    df_sorted = df.sort_values(by='dist_to_center')
    
    total_cells = len(df_sorted)
    target_num = min(num_cells, total_cells)
    if target_num < 1:
        show_info("No hay células suficientes o el número de células solicitado es inválido.")
        return
    
    distance_target = df_sorted.iloc[target_num - 1]['dist_to_center']
    circle_pts = circle_coordinates(cx=center_x, cy=center_y, radius=distance_target)

    existing_layer_names = [layer.name for layer in viewer.layers]
    final_name = shape_name
    if final_name in existing_layer_names:
        final_name += "_new"
    
    viewer.add_shapes(
        data=[circle_pts],
        shape_type='polygon',
        edge_color='yellow',
        face_color='blue',
        opacity=0.3,
        name=final_name
    )

    show_info(
        f"Círculo creado en torno a ({center_x:.2f}, {center_y:.2f}) con radio={distance_target:.2f}.\n"
        f"Total de células consideradas: {target_num} (de {total_cells})."
    )
    print(f"Círculo '{final_name}' creado. Radio aprox: {distance_target:.2f}")


# Botón para escoger el centro con un clic
pick_center_button = PushButton(label="Seleccionar centro con click")

def on_pick_center_click():
    """Se activa cuando pulsamos el botón 'Seleccionar centro con click'."""
    show_info("Haz clic en la imagen para escoger el centro...")
    print("Botón 'Seleccionar centro con click' presionado. Registrando callback...")

    def get_click(layer, event):
        """Callback que captura el primer clic y asigna (center_x, center_y)."""
        print(f"** get_click callback llamado con event={event.type}, button={event.button}")
        if event.type == 'mouse_press' and event.button == 1:
            coords_world = event.position
            coords_data = layer.world_to_data(coords_world)
            
            x_clicked, y_clicked = coords_data
            create_circle_for_n_cells.center_x.value = x_clicked
            create_circle_for_n_cells.center_y.value = y_clicked



            show_info(f"Coordenadas asignadas: X={x_clicked:.2f}, Y={y_clicked:.2f}")
            print(f"[INFO] Coordenadas asignadas: X={x_clicked:.2f}, Y={y_clicked:.2f}")

            # Quitamos el callback para no seguir capturando clics extra.
            layer.mouse_drag_callbacks.remove(get_click)
            print("Callback removido. Si quieres volver a escoger el centro, pulsa el botón otra vez.")

    # Suponiendo que la primera capa sea la de la imagen
    if len(viewer.layers) > 0:
        image_layer = viewer.layers[0]
        image_layer.mouse_drag_callbacks.append(get_click)
        print("Callback get_click ADJUNTADO a image_layer.mouse_drag_callbacks")
    else:
        show_info("No hay capas disponibles para detectar clic.")
        print("No hay capas para vincular el callback.")

pick_center_button.changed.connect(on_pick_center_click)

# Ejemplo de uso:
# 1) Selecciona el archivo con la informacion de cuantificacion para esa muestra
# 2) Nombra tu shape
# 3) Establece el numero de celulas que quieres


# Hace falta de resolver lo de el formato en el que se guarda la figura



# Add the widgets to the viewer
viewer.window.add_dock_widget(open_large_image)
#viewer.window.add_dock_widget(open_mask)
#viewer.window.add_dock_widget(load_shapes)
#viewer.window.add_dock_widget(save_contrast_limits)
#viewer.window.add_dock_widget(save_shapes)
#viewer.window.add_dock_widget(cut_mask)
#viewer.window.add_dock_widget(count_selected_cells)
#viewer.window.add_dock_widget(save_selected_cells)
#viewer.window.add_dock_widget(view_metadata)
#viewer.window.add_dock_widget(close_all)
#viewer.window.add_dock_widget(voronoi_plot)
#viewer.window.add_dock_widget(create_circle_for_n_cells)
viewer.window.add_dock_widget(pick_center_button)
